In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

In [2]:
ds = pd.read_csv("gas_turbines.csv")
ds

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
0,6.8594,1007.9,96.799,3.5000,19.663,1059.2,550.00,114.70,10.605,3.1547,82.722
1,6.7850,1008.4,97.118,3.4998,19.728,1059.3,550.00,114.72,10.598,3.2363,82.776
2,6.8977,1008.8,95.939,3.4824,19.779,1059.4,549.87,114.71,10.601,3.2012,82.468
3,7.0569,1009.2,95.249,3.4805,19.792,1059.6,549.99,114.72,10.606,3.1923,82.670
4,7.3978,1009.7,95.150,3.4976,19.765,1059.7,549.98,114.72,10.612,3.2484,82.311
...,...,...,...,...,...,...,...,...,...,...,...
15034,9.0301,1005.6,98.460,3.5421,19.164,1049.7,546.21,111.61,10.400,4.5186,79.559
15035,7.8879,1005.9,99.093,3.5059,19.414,1046.3,543.22,111.78,10.433,4.8470,79.917
15036,7.2647,1006.3,99.496,3.4770,19.530,1037.7,537.32,110.19,10.483,7.9632,90.912
15037,7.0060,1006.8,99.008,3.4486,19.377,1043.2,541.24,110.74,10.533,6.2494,93.227


In [3]:
ds.drop(['TIT','CDP'], axis=1, inplace=True)
ds

,AT,AP,AH,AFDP,GTEP,TAT,TEY,CO,NOX
0,6.8594,1007.9,96.799,3.5000,19.663,550.00,114.70,3.1547,82.722
1,6.7850,1008.4,97.118,3.4998,19.728,550.00,114.72,3.2363,82.776
2,6.8977,1008.8,95.939,3.4824,19.779,549.87,114.71,3.2012,82.468
3,7.0569,1009.2,95.249,3.4805,19.792,549.99,114.72,3.1923,82.670
4,7.3978,1009.7,95.150,3.4976,19.765,549.98,114.72,3.2484,82.311
...,...,...,...,...,...,...,...,...,...
15034,9.0301,1005.6,98.460,3.5421,19.164,546.21,111.61,4.5186,79.559
15035,7.8879,1005.9,99.093,3.5059,19.414,543.22,111.78,4.8470,79.917
15036,7.2647,1006.3,99.496,3.4770,19.530,537.32,110.19,7.9632,90.912
15037,7.0060,1006.8,99.008,3.4486,19.377,541.24,110.74,6.2494,93.227


In [4]:
print(ds.dtypes)
print(ds.isnull().sum())
ds.describe(include = 'all')

AT      float64
AP      float64
AH      float64
AFDP    float64
GTEP    float64
TAT     float64
TEY     float64
CO      float64
NOX     float64
dtype: object
AT      0
AP      0
AH      0
AFDP    0
GTEP    0
TAT     0
TEY     0
CO      0
NOX     0
dtype: int64


,AT,AP,AH,AFDP,GTEP,TAT,TEY,CO,NOX
count,15039.000000,15039.00000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000
mean,17.764381,1013.19924,79.124174,4.200294,25.419061,545.396183,134.188464,1.972499,68.190934
std,7.574323,6.41076,13.793439,0.760197,4.173916,7.866803,15.829717,2.222206,10.470586
min,0.522300,985.85000,30.344000,2.087400,17.878000,512.450000,100.170000,0.000388,27.765000
25%,11.408000,1008.90000,69.750000,3.723900,23.294000,542.170000,127.985000,0.858055,61.303500
50%,18.186000,1012.80000,82.266000,4.186200,25.082000,549.890000,133.780000,1.390200,66.601000
75%,23.862500,1016.90000,90.043500,4.550900,27.184000,550.060000,140.895000,2.160400,73.935500
max,34.929000,1034.20000,100.200000,7.610600,37.402000,550.610000,174.610000,44.103000,119.890000


In [5]:
ds.corr()

,AT,AP,AH,AFDP,GTEP,TAT,TEY,CO,NOX
AT,1.000000,-0.412953,-0.549432,-0.099333,-0.049103,0.338569,-0.207495,-0.088588,-0.600006
AP,-0.412953,1.000000,0.042573,0.040318,0.078575,-0.223479,0.146939,0.041614,0.256744
AH,-0.549432,0.042573,1.000000,-0.119249,-0.202784,0.010859,-0.110272,0.165505,0.143061
AFDP,-0.099333,0.040318,-0.119249,1.000000,0.744251,-0.571541,0.717995,-0.334207,-0.037299
GTEP,-0.049103,0.078575,-0.202784,0.744251,1.000000,-0.756884,0.977042,-0.508259,-0.208496
TAT,0.338569,-0.223479,0.010859,-0.571541,-0.756884,1.000000,-0.720356,0.063404,0.009888
TEY,-0.207495,0.146939,-0.110272,0.717995,0.977042,-0.720356,1.000000,-0.541751,-0.102631
CO,-0.088588,0.041614,0.165505,-0.334207,-0.508259,0.063404,-0.541751,1.000000,0.316743
NOX,-0.600006,0.256744,0.143061,-0.037299,-0.208496,0.009888,-0.102631,0.316743,1.000000


In [6]:
# moving the TEY column to the 0th position in the table
lastCol = ds.pop('TEY')
ds.insert(0 , 'TEY', lastCol)
print(ds.head(5))

#assigning predictor variables to x and response variable to y
x = ds.iloc[:,1:]
y = ds[['TEY']]

      TEY      AT      AP      AH    AFDP    GTEP     TAT      CO     NOX
0  114.70  6.8594  1007.9  96.799  3.5000  19.663  550.00  3.1547  82.722
1  114.72  6.7850  1008.4  97.118  3.4998  19.728  550.00  3.2363  82.776
2  114.71  6.8977  1008.8  95.939  3.4824  19.779  549.87  3.2012  82.468
3  114.72  7.0569  1009.2  95.249  3.4805  19.792  549.99  3.1923  82.670
4  114.72  7.3978  1009.7  95.150  3.4976  19.765  549.98  3.2484  82.311


### Standardizing only predictor variable - after train test split

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.20, random_state=42)

scaler_train = StandardScaler()
scaler_test = StandardScaler()

x_train_scaled = scaler_train.fit_transform(x_train) # scaling train data -- predictor
x_test_scaled  = scaler_test.fit_transform(x_test) # scaling test data -- predictor

print(x_train_scaled.shape)
print(x_test_scaled.shape)
print(y_train.shape)
print(y_test.shape)

#for removing heading from y_test
y_test = y_test.values
#print(x_train_scaled)
#print(y_test)

(12031, 8)
(3008, 8)
(12031, 1)
(3008, 1)


In [8]:
# since we have continuous ouput, AF is not required in the o/p layer
model = Sequential()
model.add( Dense( units = 50 , activation = 'relu' , kernel_initializer = 'normal', input_dim = 8)) # input layer
model.add( Dense( units = 20 , activation = 'tanh' , kernel_initializer = 'normal' )) # hidden layer
model.add( Dense( units = 1  , kernel_initializer = 'normal' )) # o/p layer

model.compile(optimizer ='adam', loss = 'mean_squared_error', metrics=['MeanSquaredError'])
model.fit(x_train_scaled, y_train , batch_size=50, epochs=100,  verbose=1)

Epoch 1/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 1s 625us/step - MeanSquaredError: 17767.9355 - loss: 17767.9062
Epoch 2/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 592us/step - MeanSquaredError: 15937.9854 - loss: 15937.9844
Epoch 3/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 567us/step - MeanSquaredError: 14685.9805 - loss: 14685.9688
Epoch 4/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 586us/step - MeanSquaredError: 13453.0908 - loss: 13453.0918
Epoch 5/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 600us/step - MeanSquaredError: 12417.0625 - loss: 12417.0586
Epoch 6/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 582us/step - MeanSquaredError: 11381.0293 - loss: 11381.0293
Epoch 7/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 602us/step - MeanSquaredError: 10410.9502 - loss: 10410.9434
Epoch 8/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 588us/step - MeanSquaredError: 9530.3164 - loss: 9530.3057
Epoch 9/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 592us/step - MeanSquaredError: 8681.6279 - loss: 8681.6260
Epoch 10/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 596us/step - M

241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 621us/step - MeanSquaredError: 1.3483 - loss: 1.3483
Epoch 62/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 625us/step - MeanSquaredError: 1.1700 - loss: 1.1700
Epoch 63/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 659us/step - MeanSquaredError: 1.0088 - loss: 1.0088
Epoch 64/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 600us/step - MeanSquaredError: 1.0157 - loss: 1.0158
Epoch 65/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 618us/step - MeanSquaredError: 0.8983 - loss: 0.8983
Epoch 66/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 618us/step - MeanSquaredError: 0.8916 - loss: 0.8916
Epoch 67/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 623us/step - MeanSquaredError: 0.8631 - loss: 0.8631
Epoch 68/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 620us/step - MeanSquaredError: 0.8252 - loss: 0.8252
Epoch 69/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 606us/step - MeanSquaredError: 0.7423 - loss: 0.7423
Epoch 70/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 613us/step - MeanSquaredError: 0.7200 - loss: 0.7200
Epoch 71/100
241/241 ━━━━━━━━━━━━

In [9]:
def toFindBestParams(x_train_scaled, y_train, x_test_scaled, y_test):
        
    #defining list of hyperparameters
    batch_size_list = [5 , 10 , 15 , 20]
    epoch_list      = [5 , 10 , 50 , 100]
     
    bestParamTable = pd.DataFrame()
    
    for batch_trial in batch_size_list:
        for epochs_trial in epoch_list:
            
            # create ANN model
            model = Sequential()
            # Defining the first layer of the model
            model.add(Dense(units=50, input_dim=x_train_scaled.shape[1], kernel_initializer='normal', activation='relu'))
            
            # Defining the Second layer of the model
            model.add(Dense(units=20, kernel_initializer='normal', activation='tanh'))
 
            # The output neuron is a single fully connected node 
            # Since we will be predicting a single number
            model.add(Dense(1, kernel_initializer='normal'))
 
            # Compiling the model
            model.compile(optimizer ='adam', loss = 'mean_squared_error')
            
            # Fitting the ANN to the Training set
            model.fit(x_train_scaled, y_train , batch_size=batch_trial, epochs=epochs_trial,  verbose=0)
                        
            MAPE = np.mean(100 * (np.abs(y_test-model.predict(x_test_scaled))/y_test))  
                        
            bestParamTable=bestParamTable.append(pd.DataFrame(data=[[batch_trial, epochs_trial, 100-MAPE]],
                                                        columns=['batchsize','epochs','Accuracy'] ))
            
            # printing the results of the current iteration
            print('batch_size:', batch_trial,'-', 'epochs:',epochs_trial, 'Accuracy:',100-MAPE)

    return bestParamTable

# Calling the function
finalParamTable_1 = toFindBestParams(x_train_scaled, y_train, x_test_scaled, y_test)
finalParamTable_1

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 591us/step
batch_size: 5 - epochs: 5 Accuracy: 98.52522309519256
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 506us/step
batch_size: 5 - epochs: 10 Accuracy: 99.47992482483473
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 546us/step
batch_size: 5 - epochs: 50 Accuracy: 99.58955211438598
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 634us/step
batch_size: 5 - epochs: 100 Accuracy: 99.6082930153929
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 641us/step
batch_size: 10 - epochs: 5 Accuracy: 81.12507347584375
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 624us/step
batch_size: 10 - epochs: 10 Accuracy: 98.7766093359925
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 699us/step
batch_size: 10 - epochs: 50 Accuracy: 99.5345198251479
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
batch_size: 10 - epochs: 100 Accuracy: 99.5623200272616
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 537us/step
batch_size: 15 - epochs: 5 Accuracy: 58.92387268601717
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 585us/step
batch_size: 15 - epochs: 10 Accuracy: 89.85945837953977
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 644us/step


,batchsize,epochs,Accuracy
0,5,5,98.525223
0,5,10,99.479925
0,5,50,99.589552
0,5,100,99.608293
0,10,5,81.125073
0,10,10,98.776609
0,10,50,99.534520
0,10,100,99.562320
0,15,5,58.923873
0,15,10,89.859458


In [10]:
# getting corresponding row values of the maximum value of 'Accuracy' column
finalParamTable_1 = finalParamTable_1.reset_index()
#print(finalParamTable_1)
#print(finalParamTable['Accuracy'].idxmax())
finalParamTable_1.iloc[finalParamTable_1['Accuracy'].idxmax()]

index          0.000000
batchsize      5.000000
epochs       100.000000
Accuracy      99.608293
Name: 3, dtype: float64

## Training Model - using best params 

In [11]:
model.compile(optimizer ='adam', loss = 'mean_squared_error')
# fitting the model to best params
model.fit(x_train_scaled,y_train, batch_size=20 , epochs = 100, verbose=0)

In [12]:
# generating predictions for test data
y_predict_test = model.predict(x_test_scaled) 

# creating table with test price & predicted price for test
final_table = pd.DataFrame(x_test)
final_table['Price'] = y_test
final_table['Predicted Price'] = y_predict_test
print(final_table.shape)
final_table.head(10)

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 823us/step
(3008, 10)


,AT,AP,AH,AFDP,GTEP,TAT,CO,NOX,Price,Predicted Price
13312,24.0930,1010.2,75.821,4.0023,25.762,550.25,1.26430,60.954,134.46,134.573898
12627,20.4500,1014.4,90.471,3.2106,20.085,549.94,2.69370,56.658,111.88,112.686584
6393,20.2620,1012.5,82.892,4.5325,25.221,549.62,1.96250,64.937,133.72,134.042786
4990,26.8620,1012.3,70.267,4.4266,25.965,549.96,1.57120,64.836,133.79,134.094818
12462,19.4090,1006.3,90.525,3.1241,19.688,550.01,2.29960,58.706,110.77,111.484459
7405,2.2158,1013.1,88.695,3.3709,19.387,550.17,4.27640,99.759,113.32,113.738152
10993,23.8520,1002.2,84.186,3.7871,25.392,550.11,0.83578,59.426,133.77,133.781769
9488,12.3950,1019.5,95.381,4.2837,23.225,549.60,2.01980,75.260,128.98,129.835068
14201,12.3590,1022.7,82.295,5.1559,32.518,528.98,0.87760,66.416,159.42,160.687744
9757,8.6376,1001.6,98.271,5.9309,32.105,530.69,10.75000,102.130,161.86,161.905365


In [13]:
# Computing the absolute percent error
APE=100*(abs(final_table['Price']-final_table['Predicted Price'])/final_table['Price'])
print('The Accuracy for Test Data -- ANN model = ', 100-np.mean(APE))

# adding absolute percent error to table
final_table['APE']=APE
final_table.head()

The Accuracy for Test Data -- ANN model =  99.6107855940776


,AT,AP,AH,AFDP,GTEP,TAT,CO,NOX,Price,Predicted Price,APE
13312,24.093,1010.2,75.821,4.0023,25.762,550.25,1.2643,60.954,134.46,134.573898,0.084708
12627,20.450,1014.4,90.471,3.2106,20.085,549.94,2.6937,56.658,111.88,112.686584,0.720937
6393,20.262,1012.5,82.892,4.5325,25.221,549.62,1.9625,64.937,133.72,134.042786,0.241389
4990,26.862,1012.3,70.267,4.4266,25.965,549.96,1.5712,64.836,133.79,134.094818,0.227833
12462,19.409,1006.3,90.525,3.1241,19.688,550.01,2.2996,58.706,110.77,111.484459,0.644993


### Standardizing both Predictor & Response variable  - before train test split

In [14]:
### Sandardization of data ###
scaler_x = StandardScaler()
scaler_y = StandardScaler()
 
# Storing the fit object for later reference
x_scaler_fit = scaler_x.fit(x)
y_scaler_fit = scaler_y.fit(y)
 
# Generating the standardized values of X and y
x = x_scaler_fit.transform(x)
y = y_scaler_fit.transform(y)
 
# Split the data into training and testing set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
 
# Shape of Training and Test datasets
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(12031, 8)
(12031, 1)
(3008, 8)
(3008, 1)


In [15]:
# since we have continuous ouput, AF is not required in the o/p layer
model = Sequential()
model.add( Dense( units = 50 , activation = 'relu' , kernel_initializer = 'normal', input_dim = 8)) # input layer
model.add( Dense( units = 20 , activation = 'tanh' , kernel_initializer = 'normal' )) # hidden layer
model.add( Dense( units = 1  , kernel_initializer = 'normal' )) # o/p layer

model.compile(optimizer ='adam', loss = 'mean_squared_error', metrics=['mae'])
model.fit(x_train, y_train , batch_size=50, epochs=100,  verbose=1)

Epoch 1/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 1s 722us/step - loss: 0.4862 - mae: 0.4413
Epoch 2/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 654us/step - loss: 0.0091 - mae: 0.0665
Epoch 3/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 733us/step - loss: 0.0049 - mae: 0.0490
Epoch 4/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 771us/step - loss: 0.0038 - mae: 0.0439
Epoch 5/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 808us/step - loss: 0.0035 - mae: 0.0424
Epoch 6/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 869us/step - loss: 0.0033 - mae: 0.0409
Epoch 7/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step - loss: 0.0032 - mae: 0.0403
Epoch 8/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 792us/step - loss: 0.0030 - mae: 0.0395
Epoch 9/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0027 - mae: 0.0379
Epoch 10/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 781us/step - loss: 0.0030 - mae: 0.0392
Epoch 11/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 675us/step - loss: 0.0029 - mae: 0.0388
Epoch 12/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 728us/step - loss: 0.0027 - mae

241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 896us/step - loss: 0.0021 - mae: 0.0330
Epoch 70/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step - loss: 0.0021 - mae: 0.0330
Epoch 71/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 738us/step - loss: 0.0020 - mae: 0.0327
Epoch 72/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 684us/step - loss: 0.0021 - mae: 0.0336
Epoch 73/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 791us/step - loss: 0.0022 - mae: 0.0339
Epoch 74/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 708us/step - loss: 0.0022 - mae: 0.0333
Epoch 75/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - loss: 0.0024 - mae: 0.0348
Epoch 76/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step - loss: 0.0021 - mae: 0.0330
Epoch 77/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 881us/step - loss: 0.0019 - mae: 0.0319
Epoch 78/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 741us/step - loss: 0.0021 - mae: 0.0331
Epoch 79/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 692us/step - loss: 0.0022 - mae: 0.0334
Epoch 80/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 0s 676us/step - loss: 0.0022 - mae: 

In [16]:
def toFindBestParams(x_train, y_train, x_test, y_test):
        
    #defining list of hyperparameters
    batch_size_list = [5 , 10 , 15 , 20]
    epoch_list      = [5 , 10 , 50 , 100]
    
    bestParamTable = pd.DataFrame()
    
    for batch_trial in batch_size_list:
        for epochs_trial in epoch_list:
                        
            # create ANN model
            model = Sequential()
            # Defining the first layer of the model
            model.add(Dense(units=50, input_dim=x_train.shape[1], kernel_initializer='normal', activation='relu'))
            
            # Defining the Second layer of the model
            model.add(Dense(units=20, kernel_initializer='normal', activation='tanh'))
 
            # The output neuron is 1 as o/p is continuous
            # No AF needed coz continuous output
            model.add(Dense(1, kernel_initializer='normal'))
 
            # Compiling the model
            model.compile(optimizer ='adam', loss = 'mean_squared_error')
            
            # Fitting the ANN to the Training set
            model.fit(x_train, y_train , batch_size=batch_trial, epochs=epochs_trial,  verbose=0)
                        
            MAPE = np.mean(100 * (np.abs(y_test-model.predict(x_test))/y_test))
            
            bestParamTable=bestParamTable.append(pd.DataFrame(data=[[batch_trial, epochs_trial, 100-MAPE]],
                                                        columns=['batchsize','epochs','Accuracy'] ))
            
            #printing the results of the current iteration
            print('batch_size:', batch_trial,'-', 'epochs:',epochs_trial, 'Accuracy:',100-MAPE)
            
    return bestParamTable

# Calling the function
finalParamTable = toFindBestParams(x_train, y_train, x_test, y_test)
#print(finalParamTable['Accuracy'].max())
#print(finalParamTable[finalParamTable['Accuracy'].max()])
finalParamTable

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 634us/step
batch_size: 5 - epochs: 5 Accuracy: 49.50238034686596
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 551us/step
batch_size: 5 - epochs: 10 Accuracy: 70.13515544128234
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 667us/step
batch_size: 5 - epochs: 50 Accuracy: 44.037366065299686
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 581us/step
batch_size: 5 - epochs: 100 Accuracy: 52.06928358759072
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 623us/step
batch_size: 10 - epochs: 5 Accuracy: 8.470804348561671
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 559us/step
batch_size: 10 - epochs: 10 Accuracy: 45.35094609984004
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 731us/step
batch_size: 10 - epochs: 50 Accuracy: 50.759951422236014
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 542us/step
batch_size: 10 - epochs: 100 Accuracy: 71.78953000291179
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 537us/step
batch_size: 15 - epochs: 5 Accuracy: 23.688182154025142
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 621us/step
batch_size: 15 - epochs: 10 Accuracy: 33.58109575120859
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1m

,batchsize,epochs,Accuracy
0,5,5,49.502380
0,5,10,70.135155
0,5,50,44.037366
0,5,100,52.069284
0,10,5,8.470804
0,10,10,45.350946
0,10,50,50.759951
0,10,100,71.789530
0,15,5,23.688182
0,15,10,33.581096


In [17]:
# getting corresponding row values of the maximum value of 'Accuracy' column
finalParamTable = finalParamTable.reset_index()
#print(finalParamTable)
#print(finalParamTable['Accuracy'].idxmax())
finalParamTable.iloc[finalParamTable['Accuracy'].idxmax()]

index          0.00000
batchsize     10.00000
epochs       100.00000
Accuracy      71.78953
Name: 7, dtype: float64

In [18]:
# fitting the model to best params
model.compile(optimizer ='adam', loss = 'mean_squared_error')
model.fit(x_train,y_train, batch_size=10 , epochs = 50, verbose=0)

In [19]:
# generating predictions for test data
y_predict_test = model.predict(x_test) 

# scaling back test data to original data
y_test_original = y_scaler_fit.inverse_transform(y_test)

# Scaling the predicted Price data back to original price scale
y_predict_test=y_scaler_fit.inverse_transform(y_predict_test)

# scaling the test input data back to original
x_test_original = x_scaler_fit.inverse_transform(x_test)

# creating table with descaled test price & descaled predicted price for test
final_table_1 = pd.DataFrame(x_test_original)
final_table_1['Price'] = y_test_original
final_table_1['Predicted Price'] = y_predict_test
print(final_table_1.shape)
final_table_1

94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
(3008, 10)


,0,1,2,3,4,5,6,7,Price,Predicted Price
0,24.093,1010.2,75.821,4.0023,25.762,550.25,1.26430,60.954,134.46,134.465591
1,20.450,1014.4,90.471,3.2106,20.085,549.94,2.69370,56.658,111.88,112.635361
2,20.262,1012.5,82.892,4.5325,25.221,549.62,1.96250,64.937,133.72,133.805389
3,26.862,1012.3,70.267,4.4266,25.965,549.96,1.57120,64.836,133.79,133.821732
4,19.409,1006.3,90.525,3.1241,19.688,550.01,2.29960,58.706,110.77,111.272018
...,...,...,...,...,...,...,...,...,...,...
3003,21.692,1006.2,80.869,3.3145,21.936,549.87,1.42310,59.948,119.25,119.849426
3004,27.200,1012.1,58.947,4.0224,25.864,550.41,0.78249,64.662,133.74,133.920074
3005,29.073,1008.3,64.511,4.7190,29.741,541.25,1.64630,59.024,146.31,146.626572
3006,23.404,1011.8,71.443,4.8017,30.052,538.12,0.84607,67.295,150.07,149.503662


In [20]:
# Computing the absolute percent error
APE_1 = 100*(abs(final_table_1['Price']-final_table_1['Predicted Price'])/final_table_1['Price'])
print('The Accuracy for Test Data -- ANN model = ', 100-np.mean(APE_1))

# adding absolute percent error to table
final_table_1['APE'] = APE_1
final_table_1.head(10)

The Accuracy for Test Data -- ANN model =  99.63660261537171


,0,1,2,3,4,5,6,7,Price,Predicted Price,APE
0,24.0930,1010.2,75.821,4.0023,25.762,550.25,1.26430,60.954,134.46,134.465591,0.004158
1,20.4500,1014.4,90.471,3.2106,20.085,549.94,2.69370,56.658,111.88,112.635361,0.675153
2,20.2620,1012.5,82.892,4.5325,25.221,549.62,1.96250,64.937,133.72,133.805389,0.063857
3,26.8620,1012.3,70.267,4.4266,25.965,549.96,1.57120,64.836,133.79,133.821732,0.023717
4,19.4090,1006.3,90.525,3.1241,19.688,550.01,2.29960,58.706,110.77,111.272018,0.453208
5,2.2158,1013.1,88.695,3.3709,19.387,550.17,4.27640,99.759,113.32,114.539749,1.076376
6,23.8520,1002.2,84.186,3.7871,25.392,550.11,0.83578,59.426,133.77,133.639267,0.097730
7,12.3950,1019.5,95.381,4.2837,23.225,549.60,2.01980,75.260,128.98,129.237274,0.199468
8,12.3590,1022.7,82.295,5.1559,32.518,528.98,0.87760,66.416,159.42,160.064194,0.404086
9,8.6376,1001.6,98.271,5.9309,32.105,530.69,10.75000,102.130,161.86,161.118759,0.457952
